In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys

sys.path.append("../")
# from src.mesoformer.datasets.metadata import ERA5Enum
# import pandas as pd
# import numpy as np


# assert ERA5Enum.difference(list("tuv")) == set(ERA5Enum).difference(ERA5Enum(list("tuv")))
# assert ERA5Enum.intersection(list("tuv")) == set(ERA5Enum).intersection(ERA5Enum(list("tuv")))

# %timeit ERA5Enum.difference(list("tuv"))
# %timeit ERA5Enum.difference(ERA5Enum(list("tuv")))



In [2]:
from __future__ import annotations

import os
import types

import pyproj
import xarray as xr
import pyresample.kd_tree

from src.mesoformer.typing import Any
from src.mesoformer.datasets.metadata import ERA5Enum, URMAEnum, CFDatasetEnum

_test_data = "../tests/data"

urma_store = os.path.join(_test_data, "urma.zarr")
urma_dvars = list(URMAEnum)

era5_store = os.path.join(_test_data, "era5.zarr")
era5_dvars = list(ERA5Enum)

print(era5_dvars, urma_dvars, sep="\n")
URMAEnum.dvars

[ERA5Enum['geopotential'], ERA5Enum['specific_humidity'], ERA5Enum['temperature'], ERA5Enum['u_component_of_wind'], ERA5Enum['v_component_of_wind'], ERA5Enum['vertical_velocity']]
[URMAEnum['total_cloud_cover'], URMAEnum['ceiling'], URMAEnum['u_wind_component_10m'], URMAEnum['v_wind_component_10m'], URMAEnum['wind_speed_10m'], URMAEnum['wind_speed_gust'], URMAEnum['wind_direction_10m'], URMAEnum['temperature_2m'], URMAEnum['dewpoint_temperature_2m'], URMAEnum['specific_humidity_2m'], URMAEnum['surface_pressure'], URMAEnum['visibility'], URMAEnum['orography']]


,coordinates,description,levels,long_name,short_name,standard_name,type_of_level,units
member_name,,,,,,,,
CEIL,"[latitude, longitude, time]",,[0],cloud ceiling,ceil,ceiling,cloudCeiling,m
D2M,"[latitude, longitude, time]",,[2],2 meter dewpoint temperature,d2m,dewpoint_temperature_2m,heightAboveGround,K
SH2,"[latitude, longitude, time]",,[2],2 meter specific humidity,sh2,specific_humidity_2m,heightAboveGround,kg kg**-1
SP,"[latitude, longitude, time]",,[0],surface pressure,sp,surface_pressure,surface,Pa
T2M,"[latitude, longitude, time]",,[2],2 meter temperature,t2m,temperature_2m,heightAboveGround,K
TCC,"[latitude, longitude, time]",,[0],total cloud cover,tcc,total_cloud_cover,atmosphereSingleLayer,%
U10,"[latitude, longitude, time]",,[10],10 meter u wind component,u10,u_wind_component_10m,heightAboveGround,m s**-1
V10,"[latitude, longitude, time]",,[10],10 meter v wind component,v10,v_wind_component_10m,heightAboveGround,m s**-1
VIS,"[latitude, longitude, time]",,[0],visibility,vis,visibility,surface,m


In [3]:
from __future__ import annotations
import abc
import typing
from typing import TypeVar, Sequence, Generic, Mapping, TypeAlias
import pandas as pd

if typing.TYPE_CHECKING:
    Series = pd.Series
else:
    Series = Sequence

_T = TypeVar("_T", covariant=True)
SeriesOrType = Series[_T] | _T  # type: ignore


_T1 = TypeVar("_T1", bound=SeriesOrType[str])
_T2 = TypeVar("_T2", bound=SeriesOrType[int])
_T3 = TypeVar("_T3", bound=SeriesOrType[list[float]])  # type: ignore

# SequenceOrType: TypeAlias = Sequence[_T] | _T
# SequenceOrTypeT = TypeVar("SequenceOrTypeT", bound=SequenceOrType)


class MetaVar(abc.ABC):
    @property
    @abc.abstractmethod
    def vars(self) -> Mapping[str, Any]:
        ...

    @property
    def short_name(self) -> pd.Series[str]:
        return self.vars["short_name"]


class DataframeVar(MetaVar):
    def __init__(self) -> None:
        self._df = pd.DataFrame(
            {
                "short_name": ["a", "b", "c"],
            }
        )

    @property
    def vars(self) -> pd.DataFrame:
        return self._df


class OtherVar(MetaVar):
    def __init__(self) -> None:
        self._df = DataframeVar()._df.loc[0]

    @property
    def vars(self) -> pd.Series:
        return self._df


dfv = OtherVar()
dfv.vars

short_name    a
Name: 0, dtype: object

In [4]:
_ENUMS = (ERA5Enum, URMAEnum)
from typing import Sequence


def find_enums(name: str | Sequence[str]) -> Sequence[CFDatasetEnum]:
    for enum_ in _ENUMS:
        try:
            return enum_(name)
        except ValueError:
            pass
    raise ValueError(f"Could not find enum for {name}")


# find_enums("temperature_2m")
# ERA5Enum.set_.issuperset(["z"])
import numpy as np
import pandas as pd

# URMAEnum.metadata.df

In [5]:
_URMA_DATASET = xr.open_zarr(urma_store)
_URMA_DATASET

<xarray.Dataset>
Dimensions:                  (time: 2, y: 1597, x: 2345)
Coordinates:
  * time                     (time) datetime64[ns] 2019-01-02 2019-01-02T01:0...
Dimensions without coordinates: y, x
Data variables: (12/14)
    ceiling                  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    latitude                 (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    longitude                (y, x) float64 dask.array<chunksize=(1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    ...                       ...
    u_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    visibility               (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (time, y, x) float32 dask.array<chunksize=(1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [6]:
if not os.path.exists(era5_store):
    _google_store = "gs://weatherbench2/datasets/era5/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2"
    xr.open_zarr(_google_store)[era5_dvars].sel(time=_URMA_DATASET.time).to_zarr(era5_store, mode="w")

_ERA5_DATASET = xr.open_zarr(era5_store)
_ERA5_DATASET

<xarray.Dataset>
Dimensions:              (time: 2, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * latitude             (latitude) float32 90.0 89.75 89.5 ... -89.75 -90.0
  * level                (level) int64 1 2 3 5 7 10 ... 875 900 925 950 975 1000
  * longitude            (longitude) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * time                 (time) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Data variables:
    geopotential         (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (time, level, latitude, longitude) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [7]:
from src.mesoformer.datasets.metadata import Dimensions


# names = [member.name for member in Dimensions]
# pd.DataFrame(
#     [list({member.name, member.value}.union(member.standard_names)) for member in Dimensions], index=names
# ).stack()


Dimensions.intersection(["time"])

{T}

In [8]:
from src.mesoformer.datasets.core import IndependentDataset, make_independent, is_independent


def get_urma() -> xr.Dataset:
    return make_independent(_URMA_DATASET.copy())


def get_era5() -> xr.Dataset:
    return make_independent(_ERA5_DATASET.copy())


ds = get_urma()


ds

<xarray.Dataset>
Dimensions:                  (T: 2, Z: 1, Y: 1597, X: 2345)
Coordinates:
    latitude                 (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.37
    longitude                (Y, X) float64 233.7 233.7 233.8 ... 300.9 301.0
  * time                     (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
    vertical                 (Z) float64 1.013e+03
Dimensions without coordinates: T, Z, Y, X
Data variables:
    ceiling                  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    specific_humidity_2m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    surface_pressure         (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    temperature_2m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    total_cloud_cover        (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    u_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    v_wind_component_10m     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    visibility               (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_direction_10m       (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_10m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    wind_speed_gust          (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [9]:
from src.mesoformer.datasets.metadata import Coordinates, Dimensions, LON, LAT, X, Y, DIMENSIONS, LVL
from src.mesoformer.datasets.core import P0

DERIVED_SFC_COORDINATE = {Coordinates.vertical: (Coordinates.vertical.axis, [P0])}


def make_independent(ds: xr.Dataset) -> xr.Dataset:
    """insures a dependant dataset is in the correct format."""
    if is_independent(ds):
        return ds
    ds = ds.rename_dims(Dimensions.map(ds.dims))
    ds = ds.rename_vars(Coordinates.map(ds.coords))
    ds = ds.set_coords(Coordinates.intersection(ds.variables))
    ds = ds.rename_vars(Coordinates.map(ds.coords))
    # ds = ds.rename_vars(Coordinates.map(ds.coords))
    ds[LON], ds[LAT] = (ds[coord].compute() for coord in (LON, LAT))
    # ds[LON] = lon
    # ds[LAT] = lat

    # - dimension assignment
    if missing := Dimensions.difference(ds.dims):
        for dim in missing:
            ds = ds.expand_dims(dim, axis=[DIMENSIONS.index(dim)])

    # - coordinate assignment
    if missing := Coordinates.difference(ds.coords):
        assert missing == {LVL}
        ds = ds.assign_coords(DERIVED_SFC_COORDINATE)

    if ds[LAT].dims == (Y,) and ds[LON].dims == (X,):
        # 5.2. Two-Dimensional Latitude, Longitude, Coordinate
        # Variables
        # The latitude and longitude coordinates of a horizontal grid that was not defined as a Cartesian
        # product of latitude and longitude axes, can sometimes be represented using two-dimensional
        # coordinate variables. These variables are identified as coordinates by use of the coordinates
        # attribute
        lon, lat = (ds[coord].to_numpy() for coord in (LON, LAT))
        yy, xx = np.meshgrid(lat, lon, indexing="xy")

        ds = ds.assign_coords({LAT: (LAT.axis, yy), LON: (LON.axis, xx)})

    ds = ds.transpose(*DIMENSIONS)
    assert is_independent(ds)
    return ds

    return ds


# print(make_independent(_URMA_DATASET)[LVL], make_independent(_ERA5_DATASET)[X].attrs, sep="\n===\n")
# _ERA5_DATASET.rename_vars(Coordinates.map(_ERA5_DATASET.coords))
# # Coordinates.difference(_ERA5_DATASET.variables), Coordinates.difference(_URMA_DATASET.variables)


# def check_types(dims):
#     return all(isinstance(dim, Dimensions) for dim in dims)


# def check_coords(coords):
#     return all(isinstance(coord, Coordinates) for coord in coords)


# [[check_types(ds.dims), check_coords(ds.coords)] for ds in (get_urma(), get_era5())]
# list(map(type, make_independent(_ERA5_DATASET).coords))

In [10]:
from src.mesoformer.datasets.metadata import Coordinates

ds = _ERA5_DATASET.copy()
# list(_ERA5_DATASET.dims)

# ds.rename_vars(Coordinates.map(ds.coords))

Coordinates.difference(_URMA_DATASET.variables)

{vertical}

In [11]:
ds = get_era5()
ds

<xarray.Dataset>
Dimensions:              (T: 2, Z: 37, Y: 721, X: 1440)
Coordinates:
    latitude             (Y, X) float32 90.0 90.0 90.0 ... -90.0 -90.0 -90.0
  * vertical             (Z) int64 1 2 3 5 7 10 20 ... 875 900 925 950 975 1000
    longitude            (Y, X) float32 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time                 (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
Dimensions without coordinates: T, Z, Y, X
Data variables:
    geopotential         (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    specific_humidity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    temperature          (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    u_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind  (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity    (T, Z, Y, X) float32 dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [12]:
import numpy as np
from src.mesoformer.datasets.core import Mesoscale, P0

pressure = [P0, 1000, 900, 800, 700, 500, 400, 300]
ms = Mesoscale(200, 175, pressure=pressure, rate=20)


extents = ms.to_area_extents(units="m")
print(ms, extents, extents.shape, sep="\n")

Mesoscale:
  scale: [ 1.00  4.74 10.68 13.44 15.26 17.70 18.60 19.37]
    hpa: [1013.25 1000.00  900.00  800.00  700.00  500.00  400.00  300.00]
     dx: [ 200.00  948.93 2135.97 2687.91 3051.46 3539.82 3720.32 3874.94]
     dy: [ 175.00  830.32 1868.97 2351.92 2670.03 3097.35 3255.28 3390.57]
[[ -200000.          -175000.           200000.           175000.        ]
 [ -948934.88302644  -830318.02264813   948934.88302644   830318.02264813]
 [-2135968.58810532 -1868972.51459216  2135968.58810532  1868972.51459216]
 [-2687907.8291667  -2351919.35052086  2687907.8291667   2351919.35052086]
 [-3051460.91319134 -2670028.29904242  3051460.91319134  2670028.29904242]
 [-3539823.89766851 -3097345.91045995  3539823.89766851  3097345.91045995]
 [-3720323.27047881 -3255282.86166896  3720323.27047881  3255282.86166896]
 [-3874941.53424558 -3390573.84246489  3874941.53424558  3390573.84246489]]
(8, 4)


In [13]:
ds = IndependentDataset.from_zarr(urma_store, URMAEnum("t2m", "d2m"))
ds

<xarray.Dataset>
Dimensions:                  (T: 2, Z: 1, Y: 1597, X: 2345)
Coordinates:
    latitude                 (Y, X) float64 19.23 19.23 19.24 ... 54.38 54.37
    longitude                (Y, X) float64 233.7 233.7 233.8 ... 300.9 301.0
  * time                     (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
    vertical                 (Z) float64 1.013e+03
Dimensions without coordinates: T, Z, Y, X
Data variables:
    temperature_2m           (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    dewpoint_temperature_2m  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [45]:
import functools


class ReSampler:
    def __init__(
        self,
        width=80,
        height=80,
        *,
        ids: IndependentDataset,
    ) -> None:
        md = ids.metadata
        self._source_definition = pyresample.geometry.AreaDefinition(
            area_id=f"{md.title} Area Definition",
            description=md.comment,
            proj_id=md.crs.name,
            projection=md.crs,
            height=ids.y.size,
            width=ids.y.size,
            area_extent=ids.area_extent,
            # lons=lons,
            # lats=lats,
        )

        self._target_definition = functools.partial(
            pyresample.geometry.AreaDefinition,
            area_id="lambert_azimuthal_equal_area",
            description="lambert_azimuthal_equal_area",
            proj_id="lambert_azimuthal_equal_area",
            width=width,
            height=height,
        )

    def get_target_definition(
        self, latitude: float, longitude: float, area_extent: list[float]
    ) -> pyresample.geometry.AreaDefinition:
        crs = pyproj.CRS.from_cf(
            {
                "grid_mapping_name": "lambert_azimuthal_equal_area",
                "latitude_of_projection_origin": latitude,
                "longitude_of_projection_origin": longitude,
                "units": "m",
            }
        )

        return self._target_definition(projection=crs, area_extent=area_extent)


def get_grid_definition(self: IndependentDataset) -> pyresample.geometry.GridDefinition:
    lons = self.lons.to_numpy()
    if is_greenwich(ds.lons.to_numpy()):
        lons = (lons - 180) % 360 - 180
        
    return pyresample.geometry.GridDefinition(lons, self.lats)


ds = IndependentDataset.from_zarr(urma_store, URMAEnum("t2m", "d2m"))
lons = (ds.lons.to_numpy() - 180) % 360 - 180

from src.mesoformer.typing import Boolean
def is_greenwich(lons: np.ndarray) -> Boolean:
    x = lons - (lons - 180) % 180
    return np.all(x == 180.0)


print(
    is_greenwich(ds.lons.to_numpy()),
    is_greenwich(lons),
    # ds.lons.to_numpy(),
    # # ds.lons.to_numpy()[0],
    # lons,
    # np.unique(lons - (lons - 180) % 360 - 180),
    sep="\n==\n",
    # ds.lats.shape,
)
# get_grid_definition(IndependentDataset.from_zarr(urma_store, URMAEnum("t2m", "d2m")))

True
==
False


In [15]:
from src.mesoformer.typing import StrPath, Sequence, DictStrAny, Mapping, Hashable, Iterable
from src.mesoformer.datasets.metadata import MetadataMixin
from src.mesoformer.datasets.metadata import T, Z, X, Y, LON, LAT, LVL, TIME, DatasetMetadata
import functools

VariableLike = type[CFDatasetEnum] | CFDatasetEnum | Sequence[CFDatasetEnum]


class PartialAreaDefinitions:
    def __init__(
        self,
        width=256,
        height=256,
        *,
        metadata: DatasetMetadata,
        latitude: float = 25.0,
        longitude: float = 265.0,
    ) -> None:
        self._source = None

        self._target = functools.partial(
            pyresample.geometry.AreaDefinition,
            area_id="lambert_azimuthal_equal_area",
            description="lambert_azimuthal_equal_area",
            proj_id="lambert_azimuthal_equal_area",
            projection=pyproj.CRS.from_cf(
                {
                    "grid_mapping_name": "lambert_azimuthal_equal_area",
                    "latitude_of_projection_origin": latitude,
                    "longitude_of_projection_origin": longitude,
                }
            ),
            width=width,
            height=height,
        )

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self._target})"

    def __call__(self, area_extent: list[float]):
        tgt = self._target(area_extent=area_extent)
        return tgt, tgt

    def iter_definitions(
        self, area_extents: Iterable[list[float]]
    ) -> Iterable[tuple[pyresample.geometry.AreaDefinition, pyresample.geometry.AreaDefinition]]:
        for area_extent in area_extents:
            yield self(area_extent)


area_defs = PartialAreaDefinitions(
    80,
    80,
    metadata=DatasetMetadata.from_title("ERA5"),
    latitude=25.0,
    longitude=265.0,
)


for src, tgt in area_defs.iter_definitions(extents):
    print(src, tgt)

Area ID: lambert_azimuthal_equal_area
Description: lambert_azimuthal_equal_area
Projection ID: lambert_azimuthal_equal_area
Projection: {'datum': 'WGS84', 'lat_0': '25', 'lon_0': '265', 'no_defs': 'None', 'proj': 'laea', 'type': 'crs', 'units': 'm', 'x_0': '0', 'y_0': '0'}
Number of columns: 80
Number of rows: 80
Area extent: (-200000.0, -175000.0, 200000.0, 175000.0) Area ID: lambert_azimuthal_equal_area
Description: lambert_azimuthal_equal_area
Projection ID: lambert_azimuthal_equal_area
Projection: {'datum': 'WGS84', 'lat_0': '25', 'lon_0': '265', 'no_defs': 'None', 'proj': 'laea', 'type': 'crs', 'units': 'm', 'x_0': '0', 'y_0': '0'}
Number of columns: 80
Number of rows: 80
Area extent: (-200000.0, -175000.0, 200000.0, 175000.0)
Area ID: lambert_azimuthal_equal_area
Description: lambert_azimuthal_equal_area
Projection ID: lambert_azimuthal_equal_area
Projection: {'datum': 'WGS84', 'lat_0': '25', 'lon_0': '265', 'no_defs': 'None', 'proj': 'laea', 'type': 'crs', 'units': 'm', 'x_0': '

In [16]:
class ReSampler:
    def __init__(
        self,
        area_defs: PartialAreaDefinitions,
    ) -> None:
        pass

In [17]:
class MesoDataset(IndependentDataset):
    cf = types.MappingProxyType(
        {
            "geographic_crs_name": "NDFD CONUS 2.5km Lambert Conformal Conic",
            "projected_crs_name": "NDFD",
            "semi_major_axis": 6378137.0,
            "semi_minor_axis": 6356752.31424518,
            "inverse_flattening": 298.25722356301,
            "reference_ellipsoid_name": "WGS 84",
            "longitude_of_prime_meridian": 0.0,
            "prime_meridian_name": "Greenwich",
            "horizontal_datum_name": "WGS84",
            "latitude_of_projection_origin": 20.191999,
            "longitude_of_projection_origin": 238.445999,
            "false_easting": 0.0,
            "false_northing": 0.0,
        }
    )

    def __init__(self, ds: xr.Dataset, dvars: VariableLike) -> None:
        super().__init__(ds, dvars)

    #     lons, lats = self.lons.to_numpy(), self.lats.to_numpy()
    #     area_extent = [lons.min(), lats.min(), lons.max(), lats.max()]
    #     self._source_definition = pyresample.geometry.AreaDefinition(
    #         self.cf["geographic_crs_name"],
    #         "National Digital Forecast Database Grid",
    #         self.cf["projected_crs_name"],
    #         self.get_crs("lambert_conformal_conic", standard_parallel=25),
    #         self.x.size,
    #         self.y.size,
    #         area_extent=area_extent,
    #         lons=lons,
    #         lats=lats,
    #     )

    # def get_source_definition(self) -> pyresample.geometry.AreaDefinition:
    #     return self._source_definition

    # def get_target_definition(
    #     self, latitude: float, longitude: float, width: float, height: float, area_extent: list[float]
    # ) -> pyresample.geometry.AreaDefinition:
    #     crs = self.get_crs("lambert_azimuthal_equal_area", latitude=latitude, longitude=longitude)
    #     return pyresample.geometry.AreaDefinition(
    #         "target_projection",
    #         "description",
    #         None,
    #         crs,
    #         width=width,
    #         height=height,
    #         area_extent=area_extent,
    #     )

    # def get_crs(
    #     self, grid_mapping_name: str, *, latitude: float | None = None, longitude: float | None = None, **kwargs
    # ) -> pyproj.CRS:
    #     origin: dict[str, Any] = {"grid_mapping_name": grid_mapping_name, "units": "m"}
    #     if latitude is not None:
    #         origin["latitude_of_projection_origin"] = latitude

    #     if longitude is not None:
    #         origin["longitude_of_projection_origin"] = longitude

    #     return pyproj.CRS.from_cf(self.cf | origin | kwargs)

    # # @property
    # # def area_extent(self) -> list[float]:
    # #     return [
    # #         self.lons.min(),
    # #         self.lats.min(),
    # #         self.lons.max(),
    # #         self.lats.max(),
    # #     ]
    # def _resample_on_center(self, target: pyresample.geometry.AreaDefinition) -> np.ndarray:
    #     return pyresample.kd_tree.resample_nearest(self.area_definition, self.to_numpy(), target, radius_of_influence=50000)

    # def resample_on_center(
    #     self,
    #     longitude: float,
    #     latitude: float,
    #     *,
    #     width=256,
    #     height=256,
    #     dx=100,
    #     dy=100,
    #     scale_x=1,
    #     scale_y=1,
    #     units="km",
    # ):
    #     if units == "km":
    #         dx *= 1000
    #         dy *= 1000

    #     dx /= 2
    #     dy /= 2

    #     height *= scale_y
    #     width *= scale_x

    #     source = self.get_source_definition()
    #     data = self.da.to_numpy()

    #     area_extent = [-dx * scale_x, -dy * scale_y, dx * scale_x, dy * scale_y]
    #     target = self.get_target_definition(latitude, longitude, width, height, area_extent)

    #     return pyresample.kd_tree.resample_nearest(source, data, target, radius_of_influence=50000)


cf = MesoDataset.from_zarr(urma_store, URMAEnum("ceiling", "vis"))
cf

<xarray.Dataset>
Dimensions:     (T: 2, Z: 1, Y: 1597, X: 2345)
Coordinates:
    latitude    (Y, X) float64 19.23 19.23 19.24 19.24 ... 54.38 54.38 54.37
    longitude   (Y, X) float64 233.7 233.7 233.8 233.8 ... 300.9 300.9 301.0
  * time        (T) datetime64[ns] 2019-01-02 2019-01-02T01:00:00
    vertical    (Z) float64 1.013e+03
Dimensions without coordinates: T, Z, Y, X
Data variables:
    ceiling     (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
    visibility  (T, Z, Y, X) float32 dask.array<chunksize=(1, 1, 1597, 2345), meta=np.ndarray>
Attributes:
    area_definition:  None

In [18]:
ceil = cf.dvars[0]
print(ceil)
ceil.crs

ceiling


AttributeError: 'URMAEnum' object has no attribute 'crs'

In [ ]:
import matplotlib.pyplot as plt


class Dataset:
    """
    # CONUS and Northern Hemisphere Grids

    https://graphical.weather.gov/docs/ndfdSRS.htm#:~:text=The%20NDFD%20uses%20the%20World%20Geodetic%20System,1984%20%28WGS84%29%20ellipsoid%20for%20its%20horizontal%20datum.

    Grid Parameter	    CONUS 2.5km
    Number of Points	2953665
    Projection Type	    Lambert Conformal
    Shape of Earth      Sphere
    Earth Radius	    6371.2 km
    Number of Points on the parallel	2145
    Number of Points on the Meridian	1377
    Latitude1:	20.191999
    Longitude1:	238.445999
    u/v vectors relative to:	easterly/northerly
    Dx	2539.703 m
    Dy	2539.703 m
    GRIB2 grid, scan mode	64 (0100)
    Scan i/x direction	positive
    Scan j/y direction	positive
    Consecutive points in	i/x direction
    Adjacent rows scan in	same direction
    Mesh Latitude	25
    Orientation Longitude	265
    Which Pole is on the Plane	north
    Is Projection Bi-polar	no
    Tangent Latitude1	25
    Tangent Latitude2	25
    Southern Latitude	-90
    Southern Longitude	0
    """

    cf = types.MappingProxyType(
        {
            "geographic_crs_name": "NDFD CONUS 2.5km Lambert Conformal Conic",
            "projected_crs_name": "NDFD",
            "semi_major_axis": 6378137.0,
            "semi_minor_axis": 6356752.31424518,
            "inverse_flattening": 298.25722356301,
            "reference_ellipsoid_name": "WGS 84",
            "longitude_of_prime_meridian": 0.0,
            "prime_meridian_name": "Greenwich",
            "horizontal_datum_name": "WGS84",
            "latitude_of_projection_origin": 20.191999,
            "longitude_of_projection_origin": 238.445999,
            "false_easting": 0.0,
            "false_northing": 0.0,
        }
    )

    def __init__(self, ds: xr.Dataset):
        self.da = da = ds.to_array().transpose(X, Y, ...)
        self.lons = lons = (da["longitude"].to_numpy() + 180) % 360 - 180
        self.lats = lats = da["latitude"].to_numpy()

        self._source_definition = pyresample.geometry.AreaDefinition(
            self.cf["geographic_crs_name"],
            "National Digital Forecast Database Grid",
            self.cf["projected_crs_name"],
            self.get_crs("lambert_conformal_conic", standard_parallel=25),
            da[Y].size,
            da[X].size,
            area_extent=self.area_extent,
            lons=lons,
            lats=lats,
        )

    def get_source_definition(self) -> pyresample.geometry.AreaDefinition:
        return self._source_definition

    def get_target_definition(
        self, latitude: float, longitude: float, width: float, height: float, area_extent: list[float]
    ) -> pyresample.geometry.AreaDefinition:
        crs = self.get_crs("lambert_azimuthal_equal_area", latitude=latitude, longitude=longitude)
        return pyresample.geometry.AreaDefinition(
            "target_projection",
            "description",
            None,
            crs,
            width=width,
            height=height,
            area_extent=area_extent,
        )

    def get_crs(
        self, grid_mapping_name: str, *, latitude: float | None = None, longitude: float | None = None, **kwargs
    ) -> pyproj.CRS:
        origin: dict[str, Any] = {"grid_mapping_name": grid_mapping_name, "units": "m"}
        if latitude is not None:
            origin["latitude_of_projection_origin"] = latitude

        if longitude is not None:
            origin["longitude_of_projection_origin"] = longitude

        return pyproj.CRS.from_cf(self.cf | origin | kwargs)

    @property
    def area_extent(self) -> list[float]:
        return [
            self.lons.min(),
            self.lats.min(),
            self.lons.max(),
            self.lats.max(),
        ]

    def resample_on_center(
        self,
        longitude: float,
        latitude: float,
        *,
        width=256,
        height=256,
        dx=100,
        dy=100,
        scale_x=1,
        scale_y=1,
        units="km",
    ):
        if units == "km":
            dx *= 1000
            dy *= 1000

        dx /= 2
        dy /= 2

        height *= scale_y
        width *= scale_x

        source = self.get_source_definition()
        data = self.da.to_numpy()

        area_extent = [-dx * scale_x, -dy * scale_y, dx * scale_x, dy * scale_y]
        target = self.get_target_definition(latitude, longitude, width, height, area_extent)

        return pyresample.kd_tree.resample_nearest(source, data, target, radius_of_influence=50000)


dataset = Dataset(get_era5().isel(T=0, Z=0))
data = dataset.resample_on_center(longitude=-89.835, latitude=38.54)
H, W, C = data.shape

fig, axes = plt.subplots(1, C, figsize=(20, 5))

for i in range(C):
    ax = axes[i]
    ax.imshow(data[:, :, i], origin="upper", cmap="terrain")
    ax.set_xticks([])
    ax.set_yticks([])